# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse     Shipments  \
0  Jun  1 2022  1:42PM  238557        12        HH6122   
1  Jun  1 2022  1:42PM  238557        12        HH6132   
2  Jun  1 2022  1:42PM  238557        12        HH6140   
3  Jun  1 2022  1:42PM  238557        12        HH6147   
4  Jun  1 2022  1:42PM  238557        12        HH6149   
5  Jun  1 2022  1:40PM  238558        10  SONSB0001807   
6  Jun  1 2022  1:39PM  238556        12      HH-34714   
7  Jun  1 2022  1:39PM  238556        12      HH-34713   
8  Jun  1 2022  1:38PM  238555        10     MET210768   
9  Jun  1 2022  1:38PM  238555        10     MET210769   

                        Customer ShipmentStatus  
0                      Hush Hush       Released  
1                      Hush Hush       Released  
2                      Hush Hush       Released  
3                      Hush Hush       Released  
4                      Hush Hush       Released  
5  Nextsource Biotechnology, LLC       Released  
6                      Hush Hush       Released  
7                      Hush Hush       Released  
8               Methapharm, Inc.       Released  
9               Methapharm, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
48  238554       Released          1
49  238555       Released         16
50  238556       Released          2
51  238557       Released          5
52  238558       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
238554                NaN        NaN       1.0
238555                NaN        NaN      16.0
238556                NaN        NaN       2.0
238557                NaN        NaN       5.0
238558                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238318                0.0       13.0       7.0
238433                0.0        8.0      15.0
238436                0.0        0.0       2.0
238438                0.0        0.0       1.0
238443                3.0       31.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238318                  0         13         7
238433                  0          8        15
238436                  0          0         2
238438                  0          0         1
238443                  3         31         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               238318          0         13         7
1               238433          0          8        15
2               238436          0          0         2
3               238438          0          0         1
4               238443          3         31         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               238318                  13        7
1               238433                   8       15
2               238436                            2
3               238438                            1
4               238443         3        31

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                          Customer
0   Jun  1 2022  1:42PM  238557        12                         Hush Hush
5   Jun  1 2022  1:40PM  238558        10     Nextsource Biotechnology, LLC
6   Jun  1 2022  1:39PM  238556        12                         Hush Hush
8   Jun  1 2022  1:38PM  238555        10                  Methapharm, Inc.
24  Jun  1 2022  1:32PM  238554        19               AdvaGen Pharma, Ltd
25  Jun  1 2022  1:28PM  238553        16  Sartorius Stedim North America  
26  Jun  1 2022  1:16PM  238552       102          Senseonics, Incorporated
27  Jun  1 2022 12:58PM  238551        19    Granules Pharmaceuticals, Inc.
28  Jun  1 2022 12:57PM  238542        10                 ISDIN Corporation
74  Jun  1 2022 12:56PM  238548        20                 Alumier Labs Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                          Customer  \
0  Jun  1 2022  1:42PM  238557        12                         Hush Hush   
1  Jun  1 2022  1:40PM  238558        10     Nextsource Biotechnology, LLC   
2  Jun  1 2022  1:39PM  238556        12                         Hush Hush   
3  Jun  1 2022  1:38PM  238555        10                  Methapharm, Inc.   
4  Jun  1 2022  1:32PM  238554        19               AdvaGen Pharma, Ltd   
5  Jun  1 2022  1:28PM  238553        16  Sartorius Stedim North America     
6  Jun  1 2022  1:16PM  238552       102          Senseonics, Incorporated   
7  Jun  1 2022 12:58PM  238551        19    Granules Pharmaceuticals, Inc.   
8  Jun  1 2022 12:57PM  238542        10                 ISDIN Corporation   
9  Jun  1 2022 12:56PM  238548        20                 Alumier Labs Inc.   

  Completed Executing Released  
0                            5  
1                            1  
2                            2  
3                           16  
4                            1  
5                            1  
6                            1  
7                            1  
8                           46  
9                           27

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                          Customer  \
0  Jun  1 2022  1:42PM  238557        12                         Hush Hush   
1  Jun  1 2022  1:40PM  238558        10     Nextsource Biotechnology, LLC   
2  Jun  1 2022  1:39PM  238556        12                         Hush Hush   
3  Jun  1 2022  1:38PM  238555        10                  Methapharm, Inc.   
4  Jun  1 2022  1:32PM  238554        19               AdvaGen Pharma, Ltd   
5  Jun  1 2022  1:28PM  238553        16  Sartorius Stedim North America     
6  Jun  1 2022  1:16PM  238552       102          Senseonics, Incorporated   
7  Jun  1 2022 12:58PM  238551        19    Granules Pharmaceuticals, Inc.   
8  Jun  1 2022 12:57PM  238542        10                 ISDIN Corporation   
9  Jun  1 2022 12:56PM  238548        20                 Alumier Labs Inc.   

  Released Executing Completed  
0        5                      
1        1                      
2        2                      
3       16                      
4        1                      
5        1                      
6        1                      
7        1                      
8       46                      
9       27

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Jun  1 2022  1:42PM  238557        12                      Hush Hush   
1  Jun  1 2022  1:40PM  238558        10  Nextsource Biotechnology, LLC   
2  Jun  1 2022  1:39PM  238556        12                      Hush Hush   
3  Jun  1 2022  1:38PM  238555        10               Methapharm, Inc.   
4  Jun  1 2022  1:32PM  238554        19            AdvaGen Pharma, Ltd   

  Released Executing Completed  
0        5                      
1        1                      
2        2                      
3       16                      
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Jun  1 2022  1:42PM  238557        12                      Hush Hush   
1  Jun  1 2022  1:40PM  238558        10  Nextsource Biotechnology, LLC   
2  Jun  1 2022  1:39PM  238556        12                      Hush Hush   
3  Jun  1 2022  1:38PM  238555        10               Methapharm, Inc.   
4  Jun  1 2022  1:32PM  238554        19            AdvaGen Pharma, Ltd   

   Released  Executing  Completed  
0       5.0        NaN        NaN  
1       1.0        NaN        NaN  
2       2.0        NaN        NaN  
3      16.0        NaN        NaN  
4       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Jun  1 2022  1:42PM  238557        12                      Hush Hush   
1  Jun  1 2022  1:40PM  238558        10  Nextsource Biotechnology, LLC   
2  Jun  1 2022  1:39PM  238556        12                      Hush Hush   
3  Jun  1 2022  1:38PM  238555        10               Methapharm, Inc.   
4  Jun  1 2022  1:32PM  238554        19            AdvaGen Pharma, Ltd   

   Released  Executing  Completed  
0       5.0        0.0        0.0  
1       1.0        0.0        0.0  
2       2.0        0.0        0.0  
3      16.0        0.0        0.0  
4       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2862233     107.0       68.0        9.0
102         715568       3.0        0.0        0.0
12          954150      67.0        2.0        0.0
15         1430558      47.0       52.0        3.0
16         1431228       7.0        0.0        0.0
18          238522       1.0        0.0        0.0
19         1192674       7.0       13.0       12.0
20         1192586      69.0        4.0        6.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2862233     107.0       68.0        9.0
1       102   715568       3.0        0.0        0.0
2        12   954150      67.0        2.0        0.0
3        15  1430558      47.0       52.0        3.0
4        16  1431228       7.0        0.0        0.0
5        18   238522       1.0        0.0        0.0
6        19  1192674       7.0       13.0       12.0
7        20  1192586      69.0        4.0        6.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     107.0       68.0        9.0
1       102       3.0        0.0        0.0
2        12      67.0        2.0        0.0
3        15      47.0       52.0        3.0
4        16       7.0        0.0        0.0
5        18       1.0        0.0        0.0
6        19       7.0       13.0       12.0
7        20      69.0        4.0        6.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  107.0
1       102  Released    3.0
2        12  Released   67.0
3        15  Released   47.0
4        16  Released    7.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10  102    12    15   16   18    19    20
Status                                                 
Completed    9.0  0.0   0.0   3.0  0.0  0.0  12.0   6.0
Executing   68.0  0.0   2.0  52.0  0.0  0.0  13.0   4.0
Released   107.0  3.0  67.0  47.0  7.0  1.0   7.0  69.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10  102    12    15   16   18    19    20
0          Completed    9.0  0.0   0.0   3.0  0.0  0.0  12.0   6.0
1          Executing   68.0  0.0   2.0  52.0  0.0  0.0  13.0   4.0
2           Released  107.0  3.0  67.0  47.0  7.0  1.0   7.0  69.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10  102    12    15   16   18    19    20
0  Completed    9.0  0.0   0.0   3.0  0.0  0.0  12.0   6.0
1  Executing   68.0  0.0   2.0  52.0  0.0  0.0  13.0   4.0
2   Released  107.0  3.0  67.0  47.0  7.0  1.0   7.0  69.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()